In [1]:
#back propergation examples
import numpy

def matrix_factorization(R, P, Q, K, steps=5000, alpha=0.0002, beta=0.02):
    Q = Q.T
    for step in xrange(steps):
        for i in xrange(len(R)):
            for j in xrange(len(R[i])):
                if R[i][j] > 0:
                    eij = R[i][j] - numpy.dot(P[i,:],Q[:,j])
                    for k in xrange(K):
                        P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])
        eR = numpy.dot(P,Q)
        e = 0
        for i in xrange(len(R)):
            for j in xrange(len(R[i])):
                if R[i][j] > 0:
                    e = e + pow(R[i][j] - numpy.dot(P[i,:],Q[:,j]), 2)
                    for k in xrange(K):
                        e = e + (beta/2) * (pow(P[i][k],2) + pow(Q[k][j],2))
        if e < 0.001:
            break
    return P, Q.T

In [2]:
R = [
     [5,3,0,1],
     [4,0,0,1],
     [1,1,0,5],
     [1,0,0,4],
     [0,1,5,4],
    ]

R = numpy.array(R)

N = len(R)
M = len(R[0])
K = 2

P = numpy.random.rand(N,K)
Q = numpy.random.rand(M,K)

nP, nQ = matrix_factorization(R, P, Q, K)
nR = numpy.dot(nP, nQ.T)

In [3]:
print(nR)

[[ 5.01614138  2.87038792  5.15985678  0.99611146]
 [ 3.94614694  2.26179871  4.20906641  0.99632856]
 [ 1.11079848  0.71795915  4.48250269  4.96090897]
 [ 0.94378191  0.60582677  3.6388662   3.97420051]
 [ 2.27835433  1.36606296  4.87185341  4.04077141]]


In [24]:

statsFile = open("baseball2016.txt", "r")
i = 0
for line in statsFile:
    homeTeam = ""
    awayTeam = ""
    i=0
    if "KC" in line: 
        token = line.split(',')
        for str in token:


            if ((i == 3) or (i == 6)):   #find the word i want
                noQuotes = str.split('"')
                if (i == 3): awayTeam = noQuotes[1]
                if (i == 6): homeTeam = noQuotes[1]
            if (i == 9): print(awayTeam + ": " + str)
            if (i == 10): 
                print(homeTeam + ": " + str)
                print('-------')
            
            i += 1
statsFile.close()



NYN: 3
KCA: 4
-------
NYN: 2
KCA: 0
-------
MIN: 3
KCA: 4
-------
MIN: 0
KCA: 7
-------
MIN: 3
KCA: 4
-------
KCA: 2
HOU: 8
-------
KCA: 3
HOU: 2
-------
KCA: 4
HOU: 2
-------
KCA: 6
HOU: 2
-------
KCA: 4
OAK: 2
-------
KCA: 3
OAK: 5
-------
KCA: 2
OAK: 3
-------
DET: 6
KCA: 8
-------
DET: 3
KCA: 2
-------
DET: 0
KCA: 4
-------
BAL: 2
KCA: 4
-------
BAL: 8
KCA: 3
-------
BAL: 1
KCA: 6
-------
KCA: 1
ANA: 6
-------
KCA: 4
ANA: 9
-------
KCA: 2
ANA: 4
-------
KCA: 0
SEA: 1
-------
KCA: 0
SEA: 6
-------
KCA: 4
SEA: 1
-------
WAS: 2
KCA: 0
-------
WAS: 6
KCA: 7
-------
WAS: 13
KCA: 2
-------
KCA: 1
CLE: 7
-------
KCA: 7
CLE: 0
-------
KCA: 4
CLE: 5
-------
KCA: 3
NYA: 6
-------
KCA: 7
NYA: 10
-------
KCA: 7
NYA: 3
-------
KCA: 3
NYA: 7
-------
ATL: 1
KCA: 5
-------
ATL: 5
KCA: 0
-------
ATL: 2
KCA: 4
-------
BOS: 4
KCA: 8
-------
BOS: 2
KCA: 3
-------
BOS: 5
KCA: 2
-------
KCA: 4
CHA: 1
-------
KCA: 2
CHA: 1
-------
KCA: 2
CHA: 3
-------
KCA: 10
MIN: 4
-------
KCA: 7
MIN: 4
-------
KCA: 5


In [25]:

statsFile = open("baseball2016.txt", "r")
i = 0
for line in statsFile:
    homeTeam = ""
    awayTeam = ""
    i=0
    token = line.split(',')
        for str in token:


            if ((i == 3) or (i == 6)):   #find the word i want
                noQuotes = str.split('"')
                if (i == 3): awayTeam = noQuotes[1]
                if (i == 6): homeTeam = noQuotes[1]
            if (i == 9): print(awayTeam + ": " + str)
            if (i == 10): 
                print(homeTeam + ": " + str)
                print('-------')
            
            i += 1
        
statsFile.close()



NYN: 3
KCA: 4
-------
NYN: 2
KCA: 0
-------
MIN: 3
KCA: 4
-------
MIN: 0
KCA: 7
-------
MIN: 3
KCA: 4
-------
KCA: 2
HOU: 8
-------
KCA: 3
HOU: 2
-------
KCA: 4
HOU: 2
-------
KCA: 6
HOU: 2
-------
KCA: 4
OAK: 2
-------
KCA: 3
OAK: 5
-------
KCA: 2
OAK: 3
-------
DET: 6
KCA: 8
-------
DET: 3
KCA: 2
-------
DET: 0
KCA: 4
-------
BAL: 2
KCA: 4
-------
BAL: 8
KCA: 3
-------
BAL: 1
KCA: 6
-------
KCA: 1
ANA: 6
-------
KCA: 4
ANA: 9
-------
KCA: 2
ANA: 4
-------
KCA: 0
SEA: 1
-------
KCA: 0
SEA: 6
-------
KCA: 4
SEA: 1
-------
WAS: 2
KCA: 0
-------
WAS: 6
KCA: 7
-------
WAS: 13
KCA: 2
-------
KCA: 1
CLE: 7
-------
KCA: 7
CLE: 0
-------
KCA: 4
CLE: 5
-------
KCA: 3
NYA: 6
-------
KCA: 7
NYA: 10
-------
KCA: 7
NYA: 3
-------
KCA: 3
NYA: 7
-------
ATL: 1
KCA: 5
-------
ATL: 5
KCA: 0
-------
ATL: 2
KCA: 4
-------
BOS: 4
KCA: 8
-------
BOS: 2
KCA: 3
-------
BOS: 5
KCA: 2
-------
KCA: 4
CHA: 1
-------
KCA: 2
CHA: 1
-------
KCA: 2
CHA: 3
-------
KCA: 10
MIN: 4
-------
KCA: 7
MIN: 4
-------
KCA: 5
